In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from datetime import datetime
from time import time

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import SimpleRNN
from keras.layers import GRU

from tensorflow.keras.callbacks import Callback

In [ ]:
def RNN_dataset(data, n_times, n_features):
    
    X = np.zeros((len(data)-n_times, n_times, n_features))
    Y = np.zeros(len(data)-n_times)

    for i in range(len(data) - n_times):

        X[i] = data[i:n_times+i, 0:n_features]
        Y[i] = data[n_times+i, -1]
        
    return X, Y

def RNN_dataset_pred(data, n_times, n_features):
    
    X = np.zeros((len(data)-n_times, n_times, n_features))

    for i in range(len(data) - n_times):

        X[i] = data[i:n_times+i, 0:n_features]
        
    return X
    
def preprocessing(data, n_times=24, test_size=0.2):
    
    scaler = MinMaxScaler()

    scaled = scaler.fit_transform(data)

    data_f = scaled
    
    n_features = data.shape[-1] - 1

    X, Y = RNN_dataset(data_f, n_times, n_features)
    
    idxs = []

    for i in range(len(X)):

        if str(Y[i]) == 'nan':

            idxs.append(i)

        else:

            j = 0

            for item in X[i]:

                if str(item[0]) == 'nan' or str(item[1]) == 'nan':

                    #print("nan found")
                    idxs.append(i)

                    break

                j+= 1

        i += 1
        
    
    X_new = np.zeros((X.shape[0] - len(idxs), X.shape[1], X.shape[2]))
    Y_new = np.zeros(len(Y) - len(idxs))

    k = 0

    for i in range(len(X)):

        if i not in idxs:

            X_new[k] = X[i]
            Y_new[k] = Y[i]

            k += 1

    X_train, X_test, Y_train, Y_test = train_test_split(X_new, Y_new, test_size=test_size, random_state=4)
    
    print("Training set:", X_train.shape, "Test set:", X_test.shape)
    
    return X_train, X_test, Y_train, Y_test, scaler, X_new, Y_new

class PrintCrossPoint(Callback):
    
    def __init__(self):
        
        self.epoch_cross = ""
        self.epoch = 0
     
    def on_epoch_end(self, epoch, logs=None):
        
        self.epoch += 1
        
        logs = logs or {}
        
        current_train_loss = logs.get("loss")
        current_val_loss = logs.get("val_loss")
        
        if current_val_loss < current_train_loss:
            
            if self.epoch_cross == "":
                self.epoch_cross = self.epoch
                
            #self.model.stop_training = True
            
    def on_train_end(self, epoch, logs=None):
        
        print("Validation loss lower than training loss from epoch %s!" % self.epoch_cross)
        
class StopCrossPoint(Callback):
    
    def __init__(self):
    
        self.epoch = 0
     
    def on_epoch_end(self, epoch, logs=None):
        
        self.epoch += 1
        
        logs = logs or {}
        
        current_train_loss = logs.get("loss")
        current_val_loss = logs.get("val_loss")
        
        if current_val_loss < current_train_loss:
                
            print("Training loss higher than validation loss from epoch %s!" % self.epoch)
                
            self.model.stop_training = True
            
def process_cabrera(infilename, outfilename):
            
    df = pd.read_excel("Datos pH Baleares/%s.xlsx" % infilename, engine="openpyxl")

    df.index = df.Time

    df_f = df.resample('D').mean()

    df_final = df_f[["Tempertaure (ºC)", "Salinity", "DO(umol kg-1)", "pHTinsitu"]]

    df_final.columns = ["Temperature", "Salinity", "Oxygen", "PH"]

    df_final.to_csv("%s.csv" % outfilename)

# Preprocessing 

In [ ]:
df = pd.read_csv("Datos pH Baleares/Cabrera.csv")

df["Time"] = pd.to_datetime(df["Time"])

df_final = df#df[df["Time"] < datetime(2021, 12, 10)]

df_final

In [ ]:
plt.scatter(df_final["Time"], df_final["PH"], s=10)

In [ ]:
window_size = 6

data_resampled = df_final[df_final["Oxygen"].astype('str') != 'nan'][["Temperature", "Oxygen", "Salinity", "PH"]].values

X_train, X_test, Y_train, Y_test, scaler, X_scaled, Y_scaled = preprocessing(data_resampled, n_times=window_size, test_size=0.1)

In [ ]:
data_new = df_final[["Temperature", "Oxygen", "Salinity"]].values

scaled_new = scaler.min_[0:data_new.shape[-1]] + data_new * scaler.scale_[0:data_new.shape[-1]]

n_features = data_new.shape[-1]

X_to_predict = RNN_dataset_pred(scaled_new, window_size, n_features)

# Bidirectional LSTM 

In [ ]:
t0 = time()

final_history = 100

while final_history > 0.8:
    
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, min_delta=0.0000005)
    callback_2 = StopCrossPoint()     

    # design network
    model_BI_LSTM = Sequential()
    model_BI_LSTM.add(Bidirectional(LSTM(3, activation='tanh',
                                  input_shape=(X_train.shape[1], X_train.shape[2]))))
    model_BI_LSTM.add(Dense(1, activation='sigmoid'))
    model_BI_LSTM.compile(loss='mse', optimizer='adam')

    # fit network
    history_BI_LSTM = model_BI_LSTM.fit(
        X_train, 
        Y_train, 
        epochs=500,
        steps_per_epoch=10,
        #batch_size=100, 
        validation_data=(X_test, Y_test), 
        verbose=0, 
        shuffle=False,
        callbacks=[callback, callback_2]
        )
    
    final_history = history_BI_LSTM.history['loss'][-1]*100

time_elapsed_BI_LSTM =  time()-t0
epochs_used_BI_LSTM = len(history_BI_LSTM.history['loss'])

final_train_loss_BI_LSTM = (history_BI_LSTM.history['loss'][-1]*100)
final_val_loss_BI_LSTM = (history_BI_LSTM.history['val_loss'][-1]*100)

init_train_loss_BI_LSTM = (history_BI_LSTM.history['loss'][0]*100)

print("Finished in", time_elapsed_BI_LSTM, "s using", epochs_used_BI_LSTM, "epochs")

y_pred_BI_LSTM = model_BI_LSTM.predict(X_to_predict)

y_pred_noscale_BI_LSTM = (y_pred_BI_LSTM - scaler.min_[-1]) / scaler.scale_[-1]

y_noscaled = (Y_scaled - scaler.min_[-1]) / scaler.scale_[-1]

yhat_BI_LSTM = model_BI_LSTM.predict(X_scaled)

yhat_BI_LSTM_noscale = (yhat_BI_LSTM - scaler.min_[-1]) / scaler.scale_[-1]

df_to_save = df_final.loc[:, ("Time", "Temperature", "Salinity", "Oxygen", "PH")]

df_to_save["DataType"] = ["" for i in range(len(df_to_save))]

for i in range(6, len(df_to_save[6:])):

    if df_to_save["PH"][i].astype('str') == 'nan' :
                
        df_to_save["PH"][i] = y_pred_noscale_BI_LSTM[i][0]
        df_to_save["DataType"].iloc[i] = "Prediction"
        
    else:
        
        df_to_save["DataType"].iloc[i] = "Observation"

In [ ]:
#y_pred_noscale_BI_LSTM[:, 0][df_final["PH"][window_size:].astype('str') != 'nan'] = df_final["PH"][window_size:][df_final["PH"][window_size:].astype('str') != 'nan'].values

plt.figure(figsize=(16, 12))

plt.subplot(2, 2, 1)
plt.plot(history_BI_LSTM.history['loss'], label='Train loss', lw=3)
plt.plot(history_BI_LSTM.history['val_loss'], label='Validation loss', lw=3)

plt.text(15, 0.05, "a)", fontsize=40)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.xlabel("Epoch", fontsize=30, labelpad=10)
plt.ylabel("Loss (MSE)", fontsize=30, labelpad=10)

plt.text(epochs_used_BI_LSTM*0.5, init_train_loss_BI_LSTM*0.006, 
         "Final train loss: %.2f%%" % final_train_loss_BI_LSTM, fontsize=16)
plt.text(epochs_used_BI_LSTM*0.5, init_train_loss_BI_LSTM*0.005,
         "Final val loss: %.2f%%" % final_val_loss_BI_LSTM, fontsize=16)

plt.legend(fontsize=20);

plt.subplot(2, 2, 3)
plt.plot(y_noscaled, lw=3)
plt.plot(yhat_BI_LSTM_noscale, lw=3)

plt.text(-5, 8.15, "c)", fontsize=40)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.xlabel("Training data sequence", fontsize=30, labelpad=10)
plt.ylabel(r"pH$_\mathrm{T}$", fontsize=30, labelpad=10)

plt.subplot(2, 2, 2)
plt.scatter(yhat_BI_LSTM, Y_scaled, s=20)
plt.plot(np.linspace(0, 1, 100), np.linspace(0, 1, 100), color='k', lw=3)

plt.text(-0.025, 0.9, "b)", fontsize=40)

plt.xlabel("Predicted values", fontsize=30, labelpad=10)
plt.ylabel("True values", fontsize=30, labelpad=10)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.subplot(2,2,4)
#plt.scatter(df_final["Time"], df_final["PH"], s=10)

plt.scatter(df_to_save["Time"][df_to_save["DataType"] == "Observation"], 
            df_to_save["PH"][df_to_save["DataType"] == "Observation"], s=1, lw=3)

plt.scatter(df_to_save["Time"][df_to_save["DataType"] == "Prediction"], 
            df_to_save["PH"][df_to_save["DataType"] == "Prediction"], s=1, lw=3)

plt.text(datetime(2019, 10, 1), 8.155, "d)", fontsize=40)

plt.xticks(fontsize=16, rotation=30)
plt.yticks(fontsize=16)

plt.ylabel(r"pH$_\mathrm{T}$", fontsize=30)

plt.tight_layout()
plt.subplots_adjust(hspace=0.3, wspace=0.2)

#plt.savefig("bidirectional_LSTM_gap_filling.pdf", dpi=300, bbox_inches='tight')

In [ ]:
df_to_save

In [ ]:
df_to_save.to_csv("Cabrera_data_w_predictions.csv")

In [ ]:
model_BI_LSTM.save('Bidirectional_LSTM_Cabrera_final.h5')

# Plot several predictions

In [ ]:
plt.figure(figsize=(8,6))

plt.scatter(df_final["Time"], df_final["PH"], s=10)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2, min_delta=0.000001)
callback_2 = StopCrossPoint() 

N_plots = 0

while N_plots < 3:    

    t0 = time()

    # design network
    model_BI_LSTM = Sequential()
    model_BI_LSTM.add(Bidirectional(LSTM(3, activation='tanh',
                                  input_shape=(X_train.shape[1], X_train.shape[2]))))
    model_BI_LSTM.add(Dense(1, activation='sigmoid'))
    model_BI_LSTM.compile(loss='mse', optimizer='adam')

    # fit network
    history_BI_LSTM = model_BI_LSTM.fit(
        X_train, 
        Y_train, 
        epochs=500,
        steps_per_epoch=10,
        #batch_size=100, 
        validation_data=(X_test, Y_test), 
        verbose=0, 
        shuffle=False,
        callbacks=[callback, callback_2]
        )

    time_elapsed_BI_LSTM =  time()-t0
    epochs_used_BI_LSTM = len(history_BI_LSTM.history['loss'])

    final_train_loss_BI_LSTM = (history_BI_LSTM.history['loss'][-1]*100)
    final_val_loss_BI_LSTM = (history_BI_LSTM.history['val_loss'][-1]*100)

    init_train_loss_BI_LSTM = (history_BI_LSTM.history['loss'][0]*100)

    print("Finished in", time_elapsed_BI_LSTM, "s using", epochs_used_BI_LSTM, "epochs")

    y_pred_BI_LSTM = model_BI_LSTM.predict(X_to_predict)

    y_pred_noscale_BI_LSTM = (y_pred_BI_LSTM - scaler.min_[-1]) / scaler.scale_[-1]

    yhat_BI_LSTM = model_BI_LSTM.predict(X_scaled)

    yhat_BI_LSTM_noscale = (yhat_BI_LSTM - scaler.min_[-1]) / scaler.scale_[-1]
    
    if final_train_loss_BI_LSTM < 0.8:

        t = df_final["Time"][window_size:]

        t_f = t[(t>datetime(2019,11,15)) & (t<datetime(2020, 7, 1))]
        yf = y_pred_noscale_BI_LSTM[(t>datetime(2019,11,15)) & (t<datetime(2020, 7, 1))]

        plt.scatter(t_f, yf, lw=3, color='C1', s=1)

        plt.text(datetime(2019, 10, 1), 8.155, "(d)", fontsize=40)

        plt.xticks(fontsize=16, rotation=30)
        plt.yticks(fontsize=16)

        plt.ylabel(r"$pH_T$", fontsize=30)
        
        N_plots += 1


In [ ]:
x = np.arange(0, 10*np.pi, 0.0001)
y = np.sin(x)

plt.plot(x, y)

In [ ]:
window_size = 6

X = np.zeros((len(x)-window_size, window_size, 1))
Y = np.zeros(len(x)-window_size)

for i in range(len(x) - window_size):

    X[i,0:30, 0] = x[i:window_size+i]
    Y[i] = y[window_size+i]
    
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=4)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2, min_delta=0.000001)
callback_2 = StopCrossPoint()     

t0 = time()

# design network
model_BI_LSTM = Sequential()
model_BI_LSTM.add(Bidirectional(LSTM(3, activation='tanh', input_shape=(X_train.shape[1],))))
model_BI_LSTM.add(Dense(1, activation='sigmoid'))
model_BI_LSTM.compile(loss='mse', optimizer='adam')

# fit network
history_BI_LSTM = model_BI_LSTM.fit(
    X_train, 
    Y_train, 
    epochs=500,
    steps_per_epoch=10,
    #batch_size=100, 
    validation_data=(X_test, Y_test), 
    verbose=0, 
    shuffle=False,
    callbacks=[callback, callback_2]
    )